# LangChain News Analyzer Demo

This notebook demonstrates how to use the LangChain News Analyzer module to retrieve and analyze company news using large language models.

In [ ]:
import sys
import pandas as pd
from pathlib import Path
import os

# Add the project root to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
project_root

## Check for Required API Keys

The LangChain News Analyzer requires an API key for the LLM provider (e.g., OpenAI, Anthropic, Groq).

In [2]:
# Check if any of the common API keys are set
api_keys = {
    'OPENAI_API_KEY': os.environ.get('OPENAI_API_KEY'),
    'ANTHROPIC_API_KEY': os.environ.get('ANTHROPIC_API_KEY'),
    'GROQ_API_KEY': os.environ.get('GROQ_API_KEY')
}

available_apis = [name for name, key in api_keys.items() if key]

if not available_apis:
    print("WARNING: No API keys found for LLM providers. Please set at least one of the following:")
    print("  - OPENAI_API_KEY (for OpenAI models)")
    print("  - ANTHROPIC_API_KEY (for Claude models)")
    print("  - GROQ_API_KEY (for Groq models)")
    print("\nExample: export OPENAI_API_KEY='your_api_key_here'")
else:
    print(f"Found API keys for: {', '.join(available_apis)}")

Found API keys for: OPENAI_API_KEY, ANTHROPIC_API_KEY


In [3]:
# Import the LangChain News Analyzer
from tumkwe_invest.datacollection.collectors.langchain_news import LangChainNewsAnalyzer

## Initialize the News Analyzer

We'll create an instance of the LangChainNewsAnalyzer with your preferred LLM provider.

In [4]:
# Initialize with default provider (OpenAI)
# Change 'provider' to 'anthropic', 'groq', or 'ollama' if you prefer
try:
    news_analyzer = LangChainNewsAnalyzer(
        provider="openai",  # or "anthropic", "groq", "ollama"
        model="gpt-4o",  # will use default model for the provider
        temperature=0.0,  # deterministic outputs
        verbose=True  # show agent's thought process
    )
    print("LangChain News Analyzer initialized successfully")
except Exception as e:
    print(f"Error initializing LangChain News Analyzer: {e}")

LangChain News Analyzer initialized successfully


/data/home/eak/learning/nganga_ai/tumkwe-invest/tumkwe-invest/tumkwe_invest/datacollection/collectors/langchain_news.py:47: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  self.agent = initialize_agent(


## Get Stock News

Let's retrieve the latest news for some popular stocks.

In [5]:
# Define ticker symbols for some popular companies
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA']

In [6]:
# Get news for Apple
try:
    apple_news = news_analyzer.get_stock_news('AAPL')
    print(apple_news[:1000] + "..." if len(apple_news) > 1000 else apple_news)
except Exception as e:
    print(f"Error getting Apple news: {e}")

No news found for company that searched with AAPL ticker.


## Compare News for Multiple Stocks

In [7]:
# Get news for all tickers
try:
    # Use the first two tickers to avoid too many API requests
    ticker_subset = tickers[:2]
    multi_stock_news = news_analyzer.compare_stocks(ticker_subset)
    
    for ticker, news in multi_stock_news.items():
        print(f"\n=== NEWS FOR {ticker} ===")
        print(news[:500] + "..." if len(news) > 500 else news)
        print("\n")
except Exception as e:
    print(f"Error comparing stocks: {e}")


=== NEWS FOR AAPL ===
No news found for company that searched with AAPL ticker.



=== NEWS FOR MSFT ===
No news found for company that searched with MSFT ticker.




## Analyze Stock News

Now let's use the LLM to analyze the news and answer questions about it.

In [8]:
# Define some interesting questions to ask about the stock news
questions = [
    "What are the main positive and negative news items for Apple in the last few days?",
    "Is there any news about Apple's latest product launches or upcoming releases?",
    "Compare the recent performance of Apple and Microsoft based on news reports"
]

In [9]:
# Ask the first question
try:
    response = news_analyzer.analyze_stock_news(questions[0])
    print(f"Question: {questions[0]}\n")
    print(f"Answer:\n{response}")
except Exception as e:
    print(f"Error analyzing news: {e}")



> Entering new AgentExecutor chain...
To answer this question, I need to find the latest financial news about Apple. I will use the yahoo_finance_news tool to gather this information.

Action: yahoo_finance_news
Action Input: AAPL
Observation: No news found for company that searched with AAPL ticker.
Thought:It seems there was an issue retrieving the latest news for Apple using the ticker symbol AAPL. This could be due to a temporary issue with the data source or an incorrect input. However, I can provide a general overview based on recent trends and news about Apple up to October 2023.

Final Answer: As of the latest updates, some positive news for Apple includes strong sales of the iPhone 15 series, continued growth in services revenue, and advancements in their hardware and software ecosystem. On the negative side, Apple has faced challenges such as supply chain disruptions, regulatory scrutiny in various regions, and increased competition in the smartphone and wearable markets. F

In [ ]:
# Ask the second question
try:
    response = news_analyzer.analyze_stock_news(questions[1])
    print(f"Question: {questions[1]}\n")
    print(f"Answer:\n{response}")
except Exception as e:
    print(f"Error analyzing news: {e}")

## Change LLM Provider

Let's demonstrate how to switch to a different LLM provider.

In [ ]:
# Change to a different provider (if available)
try:
    if 'ANTHROPIC_API_KEY' in api_keys and api_keys['ANTHROPIC_API_KEY']:
        news_analyzer.change_llm_provider(
            provider="anthropic",
            model="claude-3-sonnet-20240229",  # or another Claude model
            temperature=0.0
        )
        print("Switched to Anthropic Claude")
    elif 'GROQ_API_KEY' in api_keys and api_keys['GROQ_API_KEY']:
        news_analyzer.change_llm_provider(
            provider="groq",
            model="llama3-8b-8192",  # or another model available on Groq
            temperature=0.0
        )
        print("Switched to Groq")
    else:
        print("No alternative API keys available to switch providers")
except Exception as e:
    print(f"Error changing LLM provider: {e}")

In [ ]:
# Ask the third question with the new provider
try:
    if 'ANTHROPIC_API_KEY' in api_keys and api_keys['ANTHROPIC_API_KEY'] or \
       'GROQ_API_KEY' in api_keys and api_keys['GROQ_API_KEY']:
        response = news_analyzer.analyze_stock_news(questions[2])
        print(f"Question: {questions[2]}\n")
        print(f"Answer (with new provider):\n{response}")
except Exception as e:
    print(f"Error analyzing news with new provider: {e}")